In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0, delimiter="\t", quoting=3)
test_data = pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip", header=0, delimiter="\t", quoting=3 )

Функция для очистки текстов от html тегов и знаков препинания, которая получает список слов и удаляет слова, которые на обучение не влияют.

In [ ]:
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
def rew_word(r_rew):
    rev_text = BeautifulSoup(r_rew).get_text()
    l = re.sub("[^a-zA-Z]", " ", rev_text)
    low_case = l.lower()
    words = low_case.split()
    stop = set(stopwords.words("english"))
    words_n = [word for word in words if not word in stop]
    return( " ".join(words_n))

Обработка текстов в обучающей выборке.

In [ ]:
def data_clean(data):
    data_clean = []
    size = data["review"].size
    for i in range( 0, size):
        if((i+1)%1000 == 0):
            print(f"Review {i+1} from {size}\n") 
        data_clean.append(rew_word(data["review"][i]))
    return data_clean

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
train_clean = data_clean(train_data)

Формирование вектора признаков из выборки, полученной в результате выполнения функции data_clean.

In [ ]:
vector = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 20000)
train_vector = vector.fit_transform(train_clean)

In [ ]:
train_vector.shape

Обучение модели методом случайного леса.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 110) 
mas_train = train_vector.toarray()
clf.fit(mas_train,train_data["sentiment"])

Очищение данных тестовой выборки.

In [ ]:
test_clean = data_clean(test_data)

In [ ]:
test_vector = vector.transform(test_clean)
test_array = test_vector.toarray()

Применение модели на тестовой выборке и запись результатов в файл.

In [ ]:
result = clf.predict(test_array)
sub = pd.DataFrame( data={"id":test_data["id"], "sentiment":result} )
sub.to_csv( "submission.csv", index=False, quoting=3 )